In [ ]:
# 导入相关库
# pip install pandas -i 源
# pip install numpy -i 源
import pandas as pd
import numpy as np# 读取数据
data = pd.read_csv('./data/USER_INFO_M.csv',encoding='gbk')
data.head(3)# 1. 去重
data_drop = data.drop_duplicates() #去重
# 2. 删除无用数据 - 星座，手机品牌，手机系统，手机型号
data_drop.drop(['CONSTELLATION_DESC','MANU_NAME','OS_DESC','MODEL_NAME'],axis=1,inplace=True)
# 3. 分组，根据用户USER_ID
data_group = data_drop.groupby("USER_ID")
# 4. 取出用户最后一个月的数据
label = data_group[['USER_ID','IS_LOST']].tail(1)
# 5. 将USER_ID设置为索引
label.set_index('USER_ID',inplace=True)# 6. 提取用户信息 - 年龄&性别&终端设备
data_f1 = data_group[['CUST_SEX','CERT_AGE','TERM_TYPE']].first()
# 7. 提取出在网时长
data_f2 = data_group['INNET_MONTH'].last()
# 8. 处理合约是否有效
# 将这三个月的合约有效的情况规整为一个数据，
# 处理如下：当三个月不全为1时，用第三个月的值减去前两个的均值；
# 当三个月的值都是为1时，取值为1.5。
# 所有取值情况为-1、-0.5、0、0.5、1、1.5。

def cal_is_agree(x):
    # 根据规则处理是否有效特征
    x = np.array(x)
    if x.sum()==3:
        return 1.5
    else:
        return x[2] - x[:2].mean()
data_f3 = pd.DataFrame(data_group['IS_AGREE'].agg(cal_is_agree))
data_f3# 9. 合约到期时间
# 取第三个月的数据作为合约计划到期时长，将空值赋值为-1，
# 不是空值的到期时间以201603为基准0，
# 每增加一个月，数据为1，例如日期为201607，那么该月份的到期时间为4，以此类推。
tmp = data_group['AGREE_EXP_DATE'].last()
tmp =(pd.to_datetime(tmp,format="%Y%m") - pd.to_datetime("2016-03-01")).dt.days/30 #时长以月为单位
data_f4 = pd.DataFrame(tmp).fillna(-1) # -1填充缺失值

data_f4# 10.信用等级：取3个月的平均值。
data_f5 = pd.DataFrame(data_group['CREDIT_LEVEL'].agg('mean'))

# 11.VIP等级：将空值赋值为0 ，同一个ID值，三个月的数值相等，则取第三个月的数值；
#三个月的数值都不相等，则第三个月的数据减去前两个月的均值。
#这样的处理可以突出用户在第三个月的变化情况。
def cal_vip_lvl(x):
    a = np.array(x)
    if a[0] == a[1] == a[2]:
        return a[2]
    else:
        return a[2] - np.mean(a[:2])
data_f6 = pd.DataFrame(data_group['VIP_LVL'].agg(cal_vip_lvl).fillna(0))
# 12.本月费用：取三个月的平均值。
data_f7 = pd.DataFrame(data_group['ACCT_FEE'].mean())
# 13.平均每次通话时长：将各种通话时长除以通话次数，得到各类的平均通话时长。
# 总通话
data_f8_1 = pd.DataFrame(data_group['CALL_DURA'].sum()/data_group['CDR_NUM'].sum(),columns=['Total_mean'])
# 本地通话
data_f8_2 = pd.DataFrame(data_group['NO_ROAM_LOCAL_CALL_DURA'].sum()/data_group['NO_ROAM_LOCAL_CDR_NUM'].sum(),columns=['Local_mean'])
# 国内长途
data_f8_3 = pd.DataFrame(data_group['NO_ROAM_GN_LONG_CALL_DURA'].sum()/data_group['NO_ROAM_GN_LONG_CDR_NUM'].sum(),columns=['GN_LONG_mean'])
# 国内漫游
data_f8_4 = pd.DataFrame(data_group['GN_ROAM_CALL_DURA'].sum()/data_group['GN_ROAM_CDR_NUM'].sum(),columns=['GN_ROAM_mean'])
# 数据拼接
data_f8 = pd.concat([data_f8_1,data_f8_2,data_f8_3,data_f8_4],axis=1).fillna(0)

# 14.其余的变量处理方式相同：同一个ID下，取三个月的平均值。
data_f9 = data_group[['NO_ROAM_CDR_NUM','P2P_SMS_CNT_UP','TOTAL_FLUX','LOCAL_FLUX','GN_ROAM_FLUX','CALL_DAYS',
                     'CALLING_DAYS','CALLED_DAYS','CALL_RING','CALLING_RING','CALLED_RING']].agg('mean')

# 对所有数据重新排序
label.sort_index(inplace=True)
data_f1.sort_index(inplace=True)
data_f2.sort_index(inplace=True)
data_f3.sort_index(inplace=True)
data_f4.sort_index(inplace=True)
data_f5.sort_index(inplace=True)
data_f6.sort_index(inplace=True)
data_f7.sort_index(inplace=True)
data_f8.sort_index(inplace=True)
data_f9.sort_index(inplace=True)
# 拼接
data_new = pd.concat([data_f1,data_f2,data_f3,data_f4,data_f5,data_f6,data_f7,data_f8,data_f9,label],axis=1)

data_new#print(data_new.isnull().sum())
# 对缺失值进行填充
data_new = data_new.fillna(method='ffill').fillna(method='bfill')# 保存数据
data_new.to_csv('./data/data_new.csv',index=True,encoding='utf-8')
print('数据保存成功')